In [1]:
import requests
from bs4 import BeautifulSoup

#### **Import the Afrobarometer dataset to create a list of the countries**
1. Import the dataset
2. Run the names through coco to uniformize country names
3. Create a series with the countries names

#### **Retrieve data from Wikipedia**
The information about elections on the wikipedia pages vary considerably in format depending on the year. 

In [33]:
years = ["2016"]
election_types = ["presidential election","municipal election", "general election", "constitutional referendum", "gubernatorial election"]
countries = ["Benin Republic","Cape Verde","Chad", "Djibouti","Equatorial Guinea","The Gambia", "Gabon", "Ghana", "Morocco", "Niger", "Nigeria",
             "Republic of Congo","São Tomé e Príncipe", "Senegal", "Seychelles", "South Africa", "Somalia", "The Gambia", "Uganda", "Zambia"]
elect_by_country = {}


for country in countries:
    for year in years:

        r = requests.get("https://en.wikipedia.org/wiki/List_of_elections_in_"+year)
        soup = BeautifulSoup(r.content)
          

        gfg = soup.find_all(lambda tag: tag.name == "span" and country in tag.text)
        if len(gfg) > 1:
            break

        elif len(gfg) == 0:
            continue

        else:
            country_tag = gfg[0]

        parent_country_tag = country_tag.find_previous()
        election_tag = parent_country_tag.find_next_sibling()

        election_type = "NONE"
        for etype in election_types:
            if len(election_tag.find_all(lambda tag : tag.name == "a" and etype in tag.text)):
                election_type = etype

        date_tag = election_tag.find_next("li")
        c = date_tag.children
        get_n = 1
        for child in c:
            if get_n == 0:
                date_text = child
            else:
                get_n-=1


        if country not in elect_by_country.keys():
            elect_by_country[country] = []

        elect_by_country[country].append((election_type,date_text))
        


In [34]:
df_2016 = pd.DataFrame(columns=["Country","Election_type","Date"])
df_2016["Date"] = pd.to_datetime(df_2016["Date"])

for key in elect_by_country:
    for t in elect_by_country[key]:
        df_2016.loc[len(df_2016.index)] = (key,t[0],t[1])

##### Now we get the rest of the data from wikipedia for the rest of the years


   

In [36]:
import pandas as pd
import country_converter as coco
df = pd.read_spss("../Afrobarometer/r7_merged.sav",usecols=["COUNTRY"])
df["COUNTRY"] = coco.convert(names=df.COUNTRY.tolist(), to='name_short', not_found = None)
countries = df["COUNTRY"]

In [37]:
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import process

election_by_country = []

years = [2013,2015,2017,2018]

df = pd.DataFrame(columns=["Country","Election_type","Date"])

for year in years:
   r_all = requests.get("https://en.wikipedia.org/wiki/List_of_elections_in_"+str(year))
   soup_all = BeautifulSoup(r_all.content)
   
   soup_first_country = soup_all.find(lambda tag: tag.name == "li" and len(list(tag.parents)) == 11 and not tag.has_attr("class"))

   soup_current_continent = soup_first_country.parent
   current_continent = soup_current_continent.previous_sibling.previous_sibling.text.split("[")[0]
   
   while (current_continent != "Africa"):
      soup_current_continent = soup_current_continent.next_sibling.next_sibling.next_sibling.next_sibling
      current_continent = soup_current_continent.previous_sibling.previous_sibling.text.split("[")[0]


   election_types = ["referendum","elections","assembly","election"]

   for soup_country in soup_current_continent:
      if "" == soup_country.text.strip():
         continue

      for etype in election_types:

         if soup_country.text.__contains__(etype) or etype == "election":
            t = soup_country.text.split(etype)[0].split()
            t_election = t[-1] + " " + etype
            t_country = t[1:-1]      
            t_date = soup_country.text.split(etype)[1]
            t_date = t_date.strip()
            if t_date.__contains__("and"):
               if len(t_date.split(" ")) == 7:
                  t_date = t_date.split(" ")[0]+str(" ")+t_date.split(" ")[1]+str(" ")+t_date.split(" ")[2]
               if len(t_date.split(" ")) == 6:
                  t_date = t_date.split(" ")[0]+str(" ")+t_date.split(" ")[1]+str(" ")+t_date.split(" ")[5]
               if len(t_date.split(" ")) == 5:
                  t_date = t_date.split(" ")[0]+str(" ")+t_date.split(" ")[3]+str(" ")+t_date.split(" ")[4]  
            break
      
      s = ""
      for text in t_country:
          s += text + " "
      s = s[:-1]

      s = s.replace("Central African", "Central African Republic")
      s = s.replace("Comorian", "Comoros")
      s = s.replace("Ivorian", "Cote d'Ivoire")

      if s == "Somaliland":
         standard_name = s
      else:
         standard_name = coco.convert(names=s, to='name_short')

      if standard_name == "not found":
         choice = process.extractOne(s,countries)
         standard_name = choice[0]

      if t_date.__contains__("–"):
         t_date = t_date.split("–")[0]+str(" ")+t_date.split("–")[1].split()[1] +str(" ")+t_date.split("–")[1].split()[2]
      
      standard_name = standard_name.strip()
      t_election = t_election.strip()
      t_date = t_date.strip()
      row = [standard_name, t_election, t_date]
      df.loc[len(df.index)] = row

Malian not found in regex
Malian not found in regex
Malian not found in regex
Swazi not found in regex


Fix by hand of the election date of the Democratic Republic of the Congo general election, 2018 30 December 2018

In [38]:
df = df.replace(', 2018 30 December 2018','30 December 2018', regex=True)
df['Date']=pd.to_datetime(df['Date'])

####**Put together and write to file the retrived data for 2013, 2015, 2016, 2017, and 2018**

In [39]:
df_final = pd.concat([df, df_2016])
df_final['Date']=pd.to_datetime(df['Date'])
df_final.to_parquet("election_dates_wiki.parquet")